<a href="https://colab.research.google.com/github/Mel-iza/Weni_XP_IA/blob/main/notebook/weni_xp_hands_on_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Exploração do problema

<img src= "https://user-images.githubusercontent.com/72058182/226016150-eaa9fde8-3ecf-47e8-aa7a-c6d60ea9c632.png">

<h3><b>Problema de negócio</b></h3> 
Como identificar se uma mensagem é de um bot ou humano? <br><br>
Dificuldade em identificar esse caso se tratando de uma situação, tanto com contexto como sem, pode ser uma tarefa complicada. <br>
Em alguns casos, a consequência disso pode ser: <br>

* Desinformação e propagação de notícias falsas;
* Perdas financeiras ou de recursos pessoais/humanos;
 


### 1. Instalação das bibliotecas e importação dos dados 


Instalação das bibliotecas que vamos utilizar:

In [3]:
!pip uninstall numpy

Found existing installation: numpy 1.24.2
Uninstalling numpy-1.24.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.9
    /usr/local/lib/python3.9/dist-packages/numpy-1.24.2.dist-info/*
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libopenblas64_p-r0-15028c96.3.21.so
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.9/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-1.24.2


In [4]:
# Instalação das bibliotecas necessárias

!pip install pandas==1.5.3
!pip install -U scikit-learn
!pip install numpy==1.24.2
!pip install scipy==1.2.1 
!pip install Unidecode==1.3.6
!pip install nltk==3.8.1
!pip install spacy==3.5.0
!python -m spacy download pt_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scipy-1.2.1.tar.gz (23.1 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This

Importação das bibliotecas e módulos

In [5]:
# Importando as bibliotecas

# Manipulação de dados
import pandas as pd
import numpy as np
import unicodedata
import unidecode
import re


# Machine Learning, vetorização e métricas
import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegressionCV


# Tratamento e manipulação de texto
import nltk
import spacy
import pt_core_news_sm
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

import warnings
warnings.simplefilter("ignore")

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


----------------------

Importando os dados

In [6]:
# Seu código aqui
# Importando o conjunto de dados



,text,label
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot
4,Pera só um instante que vou checar aqui se enc...,chatbot
...,...,...
1537,42,humano
1538,start,humano
1539,Quais são os riscos? 😷,humano
1540,Nao,humano


In [7]:
#@title resultado esperado:

data =  pd.read_csv('/content/train_dataset_1.csv')
data

,text,label
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot
4,Pera só um instante que vou checar aqui se enc...,chatbot
...,...,...
1537,42,humano
1538,start,humano
1539,Quais são os riscos? 😷,humano
1540,Nao,humano


----------------

### 2. Conhecer e pré-processar os dados


Conferindo algumas informações básicas sobre o conjunto de dados

In [8]:
# Seu código aqui
# Mostrando algumas informações básicas do conjunto de dados



(1542, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1542 entries, 0 to 1541
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1542 non-null   object
 1   label   1542 non-null   object
dtypes: object(2)
memory usage: 24.2+ KB


In [8]:
#@title resultado esperado:


print('linhas e colunas: ', data.shape)
print('\n')
data.info()

linhas e colunas:  (1542, 2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1542 entries, 0 to 1541
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1542 non-null   object
 1   label   1542 non-null   object
dtypes: object(2)
memory usage: 24.2+ KB


Conferindo algumas linhas do conjunto de dados:
- primeiras linhas
- linhas escolhidas aleatoriamente

In [ ]:
# Seu código aqui
# Mostrando as primeiras linhas do conjunto de dados




In [ ]:
#@title Resultado esperado:

data.head()

,text,label
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot
4,Pera só um instante que vou checar aqui se enc...,chatbot


In [ ]:
# seu código aqui
# Mostrando algumas linhas do conjunto de dados de forma aleatoria



In [ ]:
#@title Resultado próximo ao esperado

data.sample(10)

,text,label
1011,Não,humano
1095,"Sentir uma crise de tosse durante a madruga, p...",humano
590,Sim,humano
5,Você tem algum desses problemas de saúde?\n\n1...,chatbot
480,Do que você precisa agora? (Digite o número da...,chatbot
640,start,humano
553,16/01/1979,humano
641,Não,humano
803,ok. Obrigada!,humano
1520,start,humano


---------------

Vamos verificar a quantidade de dados nulos presente no conjunto de dados?


In [ ]:
# Seu código aqui
# Quantidade de dados nulos



In [ ]:
#@title Resultado esperado:

data.isnull().sum()

text     0
label    0
dtype: int64

E agora, como fazer com que a máquina consiga compreender esses textos?. <br>

* `padronização`
* `removeção stop-words (a, e, de, o, etc)`
* `tokenização`

---

<i>Padronização</i>

In [9]:
# Função para remover acentos e números

def remover_acentos_e_numeros(txt):
    nfkd = unicodedata.normalize('NFKD', txt)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    palavraSemAcento = re.sub(r"(@[A-Za]+)|([^A-Za-z \t])|(\w+:\/\/\S+)| ^rt|http.+?","", palavraSemAcento)
    palavraSemAcento = str(palavraSemAcento).lower()
    palavraSemAcento = re.sub(r"\d+", "", palavraSemAcento)
    palavraSemAcento = re.sub(r'  ', ' ', palavraSemAcento)
    palavraSemAcento = re.sub(r'compartilheversao para impressao comentarios',' ',
                                palavraSemAcento)
    return palavraSemAcento

In [10]:
# Seu código aqui

data['texto_padronizado'] = [remover_acentos_e_numeros(i) for i in data['text']]
data.head(10)

,text,label,texto_padronizado
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot,ola muito prazer sou o seu assistente virtual ...
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot,fechado vamos fazer assim voce passa os dados ...
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot,pra adiantar vou precisar que voce me fale qua...
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot,humm eu nao entendi desculpadigite para falar ...
4,Pera só um instante que vou checar aqui se enc...,chatbot,pera so um instante que vou checar aqui se enc...
5,Você tem algum desses problemas de saúde?\n\n1...,chatbot,voce tem algum desses problemas de saude diabe...
6,Infelizmente eu não consegui te entender. Você...,chatbot,infelizmente eu nao consegui te entender voce ...
7,Qual o seu primeiro nome?,chatbot,qual o seu primeiro nome
8,Você não possui nenhum pedido em aberto!,chatbot,voce nao possui nenhum pedido em aberto
9,Você está com algum dos sintomas abaixo?\n\n1....,chatbot,voce esta com algum dos sintomas abaixo febre ...


In [ ]:
#@title Resultado esperado:
# Aplicando a função de padronização no texto e colocando ele em uma nova coluna chamada 'text_clean'

data['texto_padronizado'] = [remover_acentos_e_numeros(i) for i in data['text']]
data.head(12)

,text,label,texto_padronizado
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot,ola muito prazer sou o seu assistente virtual ...
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot,fechado vamos fazer assim voce passa os dados ...
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot,pra adiantar vou precisar que voce me fale qua...
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot,humm eu nao entendi desculpadigite para falar ...
4,Pera só um instante que vou checar aqui se enc...,chatbot,pera so um instante que vou checar aqui se enc...
5,Você tem algum desses problemas de saúde?\n\n1...,chatbot,voce tem algum desses problemas de saude diabe...
6,Infelizmente eu não consegui te entender. Você...,chatbot,infelizmente eu nao consegui te entender voce ...
7,Qual o seu primeiro nome?,chatbot,qual o seu primeiro nome
8,Você não possui nenhum pedido em aberto!,chatbot,voce nao possui nenhum pedido em aberto
9,Você está com algum dos sintomas abaixo?\n\n1....,chatbot,voce esta com algum dos sintomas abaixo febre ...


<i>Tokenização e remoção de stop words

In [14]:
# Remover as stop words com NLTK

from nltk import download
download(['punkt', 'averaged_perceptron_tagger', 'stopwords'])


from nltk import word_tokenize
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
stop_words = stopwords.words('portuguese')
print(stop_words)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [21]:
sentence = 'Eu estou aprendendo como criar uma IA no Weni XP e está sendo'\
'uma expeiência incrível'

sentence_words = word_tokenize(sentence)
print(sentence_words)

['Eu', 'estou', 'aprendendo', 'como', 'criar', 'uma', 'IA', 'no', 'Weni', 'XP', 'e', 'está', 'sendouma', 'expeiência', 'incrível']


In [23]:
def remove_stop_words(sentence_words, stop_words):
  return ' '.join([word for word in sentence_words if \
                   word not in stop_words])

In [ ]:
remove_stop_words

In [11]:
#@title # Outra forma de remover as stop words - usando txt

stop_portugues = list(open(r'stopwords-pt.txt', encoding="UTF-8"))
stop_portugues= [i.replace('\n','') for i in stop_portugues] 
stop_portugues = [remover_acentos_e_numeros(i) for i in stop_portugues]

def remover_stop_words(x):
    return [w for w in x if w not in stop_portugues]


data['texto_padronizado'] = data['texto_padronizado'].str.split().apply(remover_stop_words).str.join(' ')
data.head(8)    

,text,label,texto_padronizado
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot,ola prazer assistente virtual papel ajudar ven...
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot,fechado vamos passa dados deixo dei cadastro p...
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot,adiantar vou precisar fale email
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot,humm entendi desculpadigite falar consultordig...
4,Pera só um instante que vou checar aqui se enc...,chatbot,pera instante vou checar encontro algum pedido...
5,Você tem algum desses problemas de saúde?\n\n1...,chatbot,algum problemas saude diabetes pressao alta pr...
6,Infelizmente eu não consegui te entender. Você...,chatbot,infelizmente consegui entender gostaria falar ...
7,Qual o seu primeiro nome?,chatbot,


In [ ]:
tfidf = TfidfVectorizer(stop_words = stop_port)

X = tfidf.fit_transform(df.text_clean)
y = (df.label)

### 3. Vamos modelar? Criação de um modelo de classificação

In [ ]:
# Definindo modelos que serão testados

modelos = {"Regressão Logística": LogisticRegressionCV(),
          "XGBoost": XGBClassifier(),

In [ ]:
# Separando o texto em features e targets

X = df.text_clean
Y = df.label

In [ ]:
# Instanciando o CountVectorizer 

tokenizer = CountVectorizer(ngram_range=(1,1))

# Tokenizando os dados 
tokenizer.fit(X)
X_mat = tokenizer.transform(X)
X_df = pd.DataFrame(X_mat.todense())

In [ ]:
# Transformando o y em número para que o modelo consiga avaliar a classe. Escolhi o valor 0 para chatbot e 1 para humano
y = y.replace(['chatbot', 'humano'], [0, 1]).astype('category')


# Cross validation para avaliar a acurácia dos modelos.
for nome, modelo in modelos.items():
    accuracy = cross_val_score(modelo, X_df, y, cv = 3, scoring='accuracy').mean() # Definindo 3 folds e fazendo a média
    print(nome, accuracy)


### 4. Como saber se nosso modelo foi bem? Interpretação de métricas.

A acurácia nos retorna os elementos classificados corretamente pelo modelo.
Ela é uma métrica interessante, pois nos traz o quanto o modelo consegue classificar corretamente.

In [ ]:
# Criando dicionário e Avaliando o modelo
dict_metrica = {}
dict_metrica["Tecnica Utilizada"] = ["Bag Of Words sem stopwords"]

# Transformando o y em número para que o modelo consiga avaliar a classe. Escolhi o valor 0 para chatbot e 1 para humano
y = y.replace(['chatbot', 'humano'], [0, 1]).astype('category')

# Cross validation para avaliar a acurácia dos modelos.
for nome, modelo in modelos.items():
    accuracy = cross_val_score(modelo, X_df, y, cv = 3, scoring='accuracy').mean() # Definindo 3 folds e fazendo a média
    dict_metrica[nome] = [round(accuracy,4)]
    
# Mostrando métricas
df_metricas = pd.DataFrame(dict_metrica)
df_metricas

###. 5. Referências

* [Documentação Scikit Learn](https://scikit-learn.org/stable/)